In [ ]:
!pip install langchain -q
!pip install openai -q
!pip install chromaDB -q
!pip install unstructured -q
!pip install tiktoken
!pip install streamlit
!pip install pyngrok
!pip install pytube -q
!apt-get install -y tesseract-ocr
!apt-get install -y poppler-utils

In [ ]:
!pip install -U unstructured

In [ ]:
pip install "unstructured[pdf]"

In [4]:
!pip install "unstructured[pdf]" -q

In [5]:
%%writefile app.py

import streamlit as st
from langchain.embeddings.openai import OpenAIEmbeddings
from langchain.vectorstores import Chroma
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.chains import ConversationalRetrievalChain
from langchain.chat_models import ChatOpenAI
from langchain.document_loaders import UnstructuredFileLoader
from langchain.document_loaders.image import UnstructuredImageLoader
from langchain.document_loaders import ImageCaptionLoader
from langchain.docstore.document import Document
from langchain.document_loaders import UnstructuredFileLoader
import os
import pytube
import openai

# Chat UI title
st.header("Upload your own file and ask questions like ChatGPT")
st.subheader('File types supported: PDF')

# File uploader in the sidebar on the left
with st.sidebar:
    # Input for OpenAI API Key
    openai_api_key = st.text_input("OpenAI API Key", type="password")

    # Check if OpenAI API Key is provided
    if not openai_api_key:
        st.info("Please add your OpenAI API key to continue.")
        st.stop()

    # Set OPENAI_API_KEY as an environment variable
    os.environ["OPENAI_API_KEY"] = openai_api_key

# Initialize ChatOpenAI model
llm = ChatOpenAI(temperature=0, model_name="gpt-3.5-turbo-0125", streaming=True)

# Load version history from the text file
def load_version_history():
    with open("version_history.txt", "r") as file:
        return file.read()

# Sidebar section for uploading files and providing a YouTube URL
with st.sidebar:
    uploaded_files = st.file_uploader("Please upload your files", accept_multiple_files=True, type=None)
    #youtube_url = st.text_input("YouTube URL")

    # Create an expander for the version history in the sidebar
    with st.sidebar.expander("**Version History**", expanded=False):
      st.write(load_version_history())

    #st.info("Please refresh the browser if you decide to upload more files to reset the session", icon="🚨")

# Check if files are uploaded or YouTube URL is provided
if uploaded_files:
    # Print the number of files uploaded or YouTube URL provided to the console
    st.write("file uploaded successfully")

    # Load the data and perform preprocessing only if it hasn't been loaded before
    if "processed_data" not in st.session_state:
        # Load the data from uploaded files
        documents = []

        if uploaded_files:
            for uploaded_file in uploaded_files:
                # Get the full file path of the uploaded file
                file_path = os.path.join(os.getcwd(), uploaded_file.name)

                # Save the uploaded file to disk
                with open(file_path, "wb") as f:
                    f.write(uploaded_file.getvalue())




                if file_path.endswith(".pdf"):
                    # Use UnstructuredFileLoader to load the PDF
                    loader = UnstructuredFileLoader(file_path)
                    loaded_documents = loader.load()

                    # Extend the main documents list with the loaded documents
                    documents.extend(loaded_documents)


        # Chunk the data, create embeddings, and save in vectorstore
        text_splitter = RecursiveCharacterTextSplitter(chunk_size=1500, chunk_overlap=150)
        document_chunks = text_splitter.split_documents(documents)


        embeddings = OpenAIEmbeddings()
        vectorstore = Chroma.from_documents(document_chunks, embeddings)

        # Store the processed data in session state for reuse
        st.session_state.processed_data = {
            "document_chunks": document_chunks,
            "vectorstore": vectorstore,
        }

    else:
        # If the processed data is already available, retrieve it from session state
        document_chunks = st.session_state.processed_data["document_chunks"]
        vectorstore = st.session_state.processed_data["vectorstore"]

    # Initialize Langchain's QA Chain with the vectorstore
    qa = ConversationalRetrievalChain.from_llm(llm, vectorstore.as_retriever())

    # Initialize chat history
    if "messages" not in st.session_state:
        st.session_state.messages = []

    # Display chat messages from history on app rerun
    for message in st.session_state.messages:
        with st.chat_message(message["role"]):
            st.markdown(message["content"])

    # Accept user input
    if prompt := st.chat_input("Ask your questions?"):
        st.session_state.messages.append({"role": "user", "content": prompt})
        with st.chat_message("user"):
            st.markdown(prompt)

        # Query the assistant using the latest chat history
        history = [
            f"{message['role']}: {message['content']}"
            for message in st.session_state.messages
        ]

        # Convert the string list to list of tuples
        chat_history = [(message.split(": ")[0], message.split(": ")[1]) for message in history]


        result = qa({
            "question": prompt,
            "chat_history": chat_history
        })

        # Display assistant response in chat message container
        with st.chat_message("assistant"):
            message_placeholder = st.empty()
            full_response = result["answer"]
            message_placeholder.markdown(full_response + "|")
        message_placeholder.markdown(full_response)
        print(full_response)
        st.session_state.messages.append({"role": "assistant", "content": full_response})

else:
    st.write("Please upload your files")

Writing app.py


In [ ]:
from pyngrok import ngrok
import time

#Set auth token
ngrok.set_auth_token("2c54S0BCzBB0ZiGw5FSS82ulUNb_7wsNuyyPb22woziTCnnXn")

# Wait for the Flask app to start
#time.sleep(2)

!nohup streamlit run app.py --server.port 5011 &
# Start ngrok tunnel
ngrok_tunnel = ngrok.connect(addr='5011', proto='http', bind_tls=True)

# Print the URL
print(' * Tunnel URL:', ngrok_tunnel.public_url)

# Keep the Colab session alive
while True:
    time.sleep(120)  # Keep the session alive for 120sec